In [1]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [4]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [5]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_enviar():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE enviado = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [6]:
convertir_docx_to_pdf()

Rebuilding cache of generated files for COM support...
Checking 00020905-0000-0000-C000-000000000046x0x8x7
Could not add module (IID('{00020905-0000-0000-C000-000000000046}'), 0, 8, 7) - <class 'AttributeError'>: module 'win32com.gen_py.00020905-0000-0000-C000-000000000046x0x8x7' has no attribute 'CLSIDToClassMap'
Done.


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.docx
✅ Convertido: 10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.docx
🗑️ Eliminado original: 10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.docx
✅ Convertido: 10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.docx
🗑️ Eliminado original: 10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.docx


...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\7754_SLCB_02-12-2025_LEON MONDAQUE ROLY_CB-2025-362.docx
✅ Convertido: 7754_SLCB_02-12-2025_LEON MONDAQUE ROLY_CB-2025-362.docx
🗑️ Eliminado original: 7754_SLCB_02-12-2025_LEON MONDAQUE ROLY_CB-2025-362.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\8014_SLCB_04-12-2025_NAZER PARADA ROBERTO DOMINGO_CB-2025-365.docx
✅ Convertido: 8014_SLCB_04-12-2025_NAZER PARADA ROBERTO DOMINGO_CB-2025-365.docx
🗑️ Eliminado original: 8014_SLCB_04-12-2025_NAZER PARADA ROBERTO DOMINGO_CB-2025-365.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\9677_SLCB_04-12-2025_OVANDO JIMENEZ JAIRO ESTEVAN_CB-2025-338.docx
✅ Convertido: 9677_SLCB_04-12-2025_OVANDO JIMENEZ JAIRO ESTEVAN_CB-2025-338.docx
🗑️ Eliminado original: 9677_SLCB_04-12-2025_OVANDO JIMENEZ JAIRO ESTEVAN_CB-2025-338.docx


In [7]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"✅[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"❌[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [8]:
contenido = os.listdir(directorio_destino)
print(len(contenido))
contenido

64


['10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.pdf',
 '10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.pdf',
 '10863_SLCB_29-11-2025_PAZ REA ROSENDO_CB-2025-332.pdf',
 '11231_SLCB_04-12-2025_RAFAEL GARNICA FIDEL_CB-2025-357.pdf',
 '11860_SLCB_29-11-2025_ROCA SERRANO ARMANDO_CB-2025-333.pdf',
 '12162_SLCB_03-12-2025_ROCA REYES MARCOS_CB-2025-376.pdf',
 '1218_SLCB_03-12-2025_BALCAZAR ALCANTARA JOSE LOLY_CB-2025-348.pdf',
 '13062_SLCB_03-12-2025_SOTO ARCE BACILIO_CB-2025-371.pdf',
 '13062_SLCB_28-11-2025_SOTO ARCE BACILIO_CB-2025-369.pdf',
 '13067_SLCB_04-12-2025_SAUCEDO CERRUTI SONIA_CB-2025-380.pdf',
 '13132_SLCB_03-12-2025_ZUMARAN MENACHO JORGE_CB-2025-375.pdf',
 '13132_SLCB_03-12-2025_ZUMARAN MENACHO JORGE_CB-2025-381.pdf',
 '13652_SLCB_03-12-2025_TENORIO CERMENO WILLIAMS_CB-2025-355.pdf',
 '13652_SLCB_03-12-2025_TENORIO CERMENO WILLIAMS_CB-2025-384.pdf',
 '13763_SLCB_03-12-2025_TUFINO SCHWENK RUBEN DARIO_CB-2025-334.pdf',
 '14760_SLCB_28-11-2025_VARGAS PINTO 

In [9]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

CB-2025-330
[INFO] El archivo '10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '10697_SLCB_29-11-2025_PAZ REA JAIME EDUARDO_CB-2025-330.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-330
✅ Archivo eliminado.
CB-2025-370
[INFO] El archivo '10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '10806_SLCB_04-12-2025_PALICIO BRUNO OSCAR ROBERTO_CB-2025-370.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-370
✅ Archivo eliminado.
CB-2025-332
[INFO] El archivo '10863_SLCB_29-11-2025_PAZ REA ROSENDO_CB-2025-332.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '10863_SLCB_29-11-2025_PAZ REA ROSENDO_CB-2025-332.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-332
✅ Archivo eliminado.
CB-2025-357
[INFO] El archivo '11231_SLCB_04-12-2025_RAFAEL GARNICA FIDEL_CB-2025-357.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '11231_S

In [11]:
import psycopg2

In [12]:
def obtener_conexion():
    return psycopg2.connect(
        host='localhost', 
        port=5433, 
        database='utea', 
        user='postgres', 
        password='A123456*')
    
def insertar_mensaje_whatsapp(cod_canero, nombre_canero, numero_contac, mensaje, enviado=False, fecha_enviado=None):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            INSERT INTO notificaciones_wsp.msj_whatsapp (
                cod_canero, nombre_canero, numero_cel, mensaje, enviado, fecha_enviado, numero_contac
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (cod_canero, nombre_canero, 0, mensaje, enviado, fecha_enviado, numero_contac))
        
        conexion.commit()
        cursor.close()
        print("Mensaje insertado correctamente.")
    except Exception as e:
        conexion.rollback()
        print("Error inesperado")
    return None

def marcar_resumen_lib_enviado(id_resumen):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            UPDATE drones_control_bio.resumen_control_bio
            SET enviado = true
            WHERE id = %s
        """, (id_resumen,))
        conexion.commit()
        cursor.close()
        print(f"Mensaje con id={id_resumen} marcado como enviado.")
    except Exception as e:
        conexion.rollback()
        print(f"Error inesperado: {e}")
    finally:
        if conexion:
            conexion.close()
    return None

In [13]:
PATH_XLSX_CONTAC=r'G:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - EQUIPO AVIACION UTEA\\Pulverizacion\\Data\\CONTACTOS.xlsx'

In [14]:
sin_enviar = get_registros_sin_enviar()

In [15]:
cods_ca = list(set(sin_enviar['id']))
print(cods_ca)

[129, 262, 269, 297, 299, 300, 302, 303, 179, 180, 181, 183, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 200, 328, 202, 217, 97, 233, 109, 111, 113, 242, 243, 116, 118, 251]


In [16]:
len(cods_ca)

37

In [17]:
df_contac = pd.read_excel(PATH_XLSX_CONTAC)
df_contac.fillna(0, inplace=True)
df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [18]:
df_contac.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,41529,ACUÑA CANO VICTORIANO,75540191,0.0,0.0
1,41594,AGROP. CAMPODULCE SRL,77166691,65004305.0,70804997.0
2,765,AGROP. CATALA S.R.L.,77396051,0.0,0.0


In [19]:
sin_enviar.head(3)

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca,enviado
0,97,2025-09-24,8,1165,BEJARANO GANDARILLA ERWIN,11.03,CB-2025-90,40,SAN VICENTE--ERWIN BEJARANO,['TRICHOGRAMMA'],https://drive.google.com/file/d/1g8YpDinDp3gkD...,True,False
1,183,2025-10-15,304,1478,CALLE RALDEZ MARCO ANTONIO,3.53,CB-2025-176,1411,SAN JORGE--RENE CALLE,['TRICHOGRAMMA'],https://drive.google.com/file/d/1jh4DaMf1wyvzh...,True,False
2,217,2025-10-24,50,7506,LOHENER ROSENDA SHEILA PAZ DE,30.58,CB-2025-210,327,EL RECREO--ROSENDA SHEILA PAZ,['TRICHOGRAMMA'],https://drive.google.com/file/d/1I2alnKGfloK3k...,True,False


### Mensaje dirigiendo a agrocittca

In [20]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'❌ Codigo: {cod} no tinene numeros validos registrados')
            continue
        
        for num in telefonos_validos:
            msj = f'''🌱 *Saludos {row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRÁ* para comunicarle que el informe del último servicio de *🐝 Control Biológico* realizado ya se encuentra disponible en *AgroCITTCA*.

📲 *Adjuntamos sus credenciales* para acceder a la App y los enlaces de descarga 👇

    👤 *Usuario:* {row['cod_canero']}
    🔑 *Contraseña:* 123*{row['cod_canero']}

⬇️ *Descargue la App:* 

📱 *Android:* https://play.google.com/store/apps/details?id=com.dima.guabira  

🍎 *iPhone:* https://apps.apple.com/us/app/agro-cittca-guabir%C3%A1/id1669149924

🚀 Una vez instalada la app, inicie sesión con su usuario y contraseña, y acceda a:  
📂 *AGROCITTCA → Seguimiento a campo → Informes*

Si tiene inconvenientes para acceder o visualizar los informes, no dude en consultarnos. ¡Gracias por su confianza! 🙌
'''         
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 1165 - BEJARANO GANDARILLA ERWIN no tiene registro telefonico
Codigo: 1478 - CALLE RALDEZ MARCO ANTONIO no tiene registro telefonico
Codigo: 7506 - LOHENER ROSENDA SHEILA PAZ DE no tiene registro telefonico
Codigo: 41562 - SUAREZ SALAS ERWIN RONALD no tiene registro telefonico
Mensaje insertado correctamente.
Mensaje con id=325 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=325 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=325 marcado como enviado.
Codigo: 1182 - BOHORQUEZ FERNANDEZ NICANOR no tiene registro telefonico
Codigo: 2353 - CARO BENITES GERMAN no tiene registro telefonico
Mensaje insertado correctamente.
Mensaje con id=326 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=326 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=326 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=317 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=317 marcado 

### Mensaje dirigiendo a PDF

In [24]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'Codigo: {cod} no tinene numeros validos registrados')
            continue
        
        for num in telefonos_validos:
            msj = f"Saludos *{row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRA* para hacer llegar una copia del informe de servicio de control biologico en caña de azucar. También informale este informe lo puede encotrar en la aplicación AgroCITTCA, y que a partir de los proximos servicos los informes se enviaran direntamente a AgroCITTCA. Adjunto informe: {row['link']}"
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 1165 - BEJARANO GANDARILLA ERWIN no tiene registro telefonico
Codigo: 16213 - ZAPATA SERON SIMON no tiene registro telefonico
Codigo: 2351 - CANDIA RUIZ ADALBERTO no tiene registro telefonico
Codigo: 18062 - MERCADO MERCADO RUBEN DARIO no tiene registro telefonico
Codigo: 13187 - SUAREZ CUELLAR REMBERTO no tiene registro telefonico
Codigo: 2565 - CANDIA RUIZ JOSE LUIS no tiene registro telefonico
Codigo: 42229 - CANDIA ROJAS JOSE LUIS no tiene registro telefonico
Codigo: 42378 - CANDIA RUIZ ORLANDO no tiene registro telefonico
Codigo: 41562 - SUAREZ SALAS ERWIN RONALD no tiene registro telefonico
